<a href="https://colab.research.google.com/github/Student-PIAIC/OpenAI-Agents/blob/main/Guardrails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq openai-agents pydentic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from pydantic import BaseModel
from agents import (
    Agent,
    RunContextWrapper,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig,
    Runner,
    TResponseInputItem,

    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    OutputGuardrailTripwireTriggered,
    input_guardrail,
    output_guardrail
)
from google.colab import userdata

In [4]:
gemini_api_key = userdata.get('gemini_api_key')
if gemini_api_key is None:
  raise ValueError("gemini_api_key is not set define it in your .env file")

In [5]:
external_client = AsyncOpenAI(
    api_key = gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config=RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

In [17]:
class MathHomeworkOutput(BaseModel):
    is_math_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking you to do their math homework.",
    output_type=MathHomeworkOutput,
)


@input_guardrail
async def math_guardrail(
    ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    # Pass the run_config to the Runner.run call within the guardrail
    result = await Runner.run(guardrail_agent, input, context=ctx.context, run_config = config)

    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=not result.final_output.is_math_homework
  )

In [18]:
result = Runner.run_sync(guardrail_agent, "what is 2+2+3", run_config=config)
result

RunResult(input='what is 2+2+3', new_items=[MessageOutputItem(agent=Agent(name='Guardrail check', instructions='Check if the user is asking you to do their math homework.', handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=<class '__main__.MathHomeworkOutput'>, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='__fake_id__', content=[ResponseOutputText(annotations=[], text='{\n  "reasoning": "The user is asking a basic arithmetic question. This is likely a math homework question.",\n  "is_math_homework": true\n}', type='output_text')], ro

In [19]:
print(result.final_output)
print(result.final_output.is_math_homework)
print(result.final_output.reasoning)

is_math_homework=True reasoning='The user is asking a basic arithmetic question. This is likely a math homework question.'
True
The user is asking a basic arithmetic question. This is likely a math homework question.


In [20]:
agent = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    input_guardrails=[math_guardrail],)

In [24]:
"""This code tries to run an AI agent with user input and print the result
 if a guardrail is triggered
 it catches the error and shows a warning message instead."""

try:
    result = await Runner.run(agent, "Hello, tell me about pakistan?", run_config = config)
    print("Guardrail didn't trip - this is unexpected")
    print(result.final_output)

except InputGuardrailTripwireTriggered:
    print("Math homework guardrail tripped")

Math homework guardrail tripped


In [22]:

try:
    result = await Runner.run(agent, "Hello, can you help me solve for x: 2x + 3 = 11?", run_config = config)
    print("Guardrail didn't trip - this is unexpected")
    print(result.final_output)

except InputGuardrailTripwireTriggered:
    print("defence guardrail tripped")

Guardrail didn't trip - this is unexpected
Hi there! I can definitely help you with that. Here's how to solve for x in the equation 2x + 3 = 11:

1.  **Subtract 3 from both sides of the equation:**
    2x + 3 - 3 = 11 - 3
    This simplifies to:
    2x = 8

2.  **Divide both sides of the equation by 2:**
    2x / 2 = 8 / 2
    This simplifies to:
    x = 4

Therefore, the solution is **x = 4**.

Is there anything else I can help you with today?

